# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [15]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = root_url + "/status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{status_url}")
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(f"OK, connected: {req.status_code}")
else:
     print(f"The connection status code is: {req.status_code}")

OK, connected: 200


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [16]:
# Set the countries_url variable (1 line)
countries_url = root_url + "/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{countries_url}")

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(f"The req = 'countries' status code reply is: {req.status_code}, variable 'countries' return: {countries} ")

The req = 'countries' status code reply is: 403, variable 'countries' return: {'message': 'The cookie is missing'} 


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [17]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + '/cookie'
# Query the enpoint, set the cookies variable and display it (2 lines)
print(f"Status code for cookies: {requests.get(f'{cookie_url}').status_code}")

cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
print(cookies_val)

Status code for cookies: 200
196c6a5f-9ef8-404b-9c71-0ab43880a936


Try to query the countries endpoint using the cookie, save the output and print it.

In [18]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()
# display the countries variable (1 line)
print(countries)

['be', 'us', 'ma', 'fr', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [19]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + '/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies = {"user_cookie": cookies_val})
# display the leaders variable (1 line)
print(f"Type 'leaders.text': {type(leaders.text)}, leaders.text: {leaders.text}")


Type 'leaders.text': <class 'str'>, leaders.text: {"message":"Please specify a country"}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [20]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
import random
country = random.choice(countries)

payload = {'country': country}

# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = payload).json()

# display the leaders variable (1 line)
print(leaders)

[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https://nl

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [21]:
# 4 lines
leaders_per_country = {}
for country in countries:
    leaders = requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country})
    leaders_per_country[country] = leaders.json()
print(leaders_per_country)

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

In [ ]:
# or 1 line
leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}
print(leaders_per_country)

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [22]:
# < 15 lines
def get_leaders() -> dict:
    # assign root_url and endpoints
    root_url = "https://country-leaders.onrender.com"
    status_url = root_url + "/status"
    countries_url = root_url + "/countries"
    leaders_url = root_url + '/leaders'
    cookie_url = root_url + '/cookie'

    # query the /status endpoint using the get() method and store it in the req variable (1 line)
    req = requests.get(f"{status_url}")
    
    # Query the cookies enpoint, set the cookies variable and display it (2 lines)
    cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
    print('User cookie value:', cookies_val)

    # query the /countries endpoint, assign the output to the countries variable (1 line)
    countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()

    # display the request's status code and the countries variable (1 line)
    print(f"The request for 'countries' status code reply is: {req.status_code}, variable 'countries' return: {countries} ")

    leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}
    
    return leaders_per_country

get_leaders()


User cookie value: 47006e5f-ff14-45e2-87d6-0421ef6a72ce
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'None',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27',
   'd

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [23]:
# 2 lines
president_list = []
leaders_per_country = get_leaders()
for k, v in leaders_per_country.items():
    for n in v:
        president_list.append(f"{k} {n['last_name']} {n['first_name']} {n['id']} {n['wikipedia_url']}")
#print(leaders_per_country)
for h in president_list:
    print(h)

User cookie value: a731ae1b-096c-4d86-aca0-e7d03cabc4a0
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 
be Verhofstadt Guy Q12978 https://nl.wikipedia.org/wiki/Guy_Verhofstadt
be Leterme Yves Q12981 https://nl.wikipedia.org/wiki/Yves_Leterme
be None Herman Q12983 https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
be Delacroix Léon Q14989 https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix
be Carton Henry Q14990 https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart
be Theunis Georges Q14991 https://nl.wikipedia.org/wiki/Georges_Theunis
be Van de Vyvere Aloïs Q14992 https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
be Poullet Prosper Q14993 https://nl.wikipedia.org/wiki/Prosper_Poullet
be Jaspar Henri Q14994 https://nl.wikipedia.org/wiki/Henri_Jaspar
be None Jules Q14995 https://nl.wikipedia.org/wiki/Jules_Renkin
be Van Zeeland Paul Q14996 https://nl.wikipedia.org/wiki/Paul_van_Zeeland
be Van Acker Achille Q14997 https://nl.wik

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [24]:
# 3 lines
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=president_list,
    description='Leader:'
)

button = widgets.Button(description="Confirm:")

def on_button_click(b):
    global url_president, wiki_req
    selected_value = dropdown.value
    selected_list = selected_value.rsplit()
    url_president = selected_list[-1]
    #print(url_president)
    wiki_req = requests.get(url_president, headers = {'User-Agent': 'wiki_scraper'}).text


button.on_click(on_button_click)

display(dropdown, button)

#url_president = next(item['wikipedia_url'] for item in leaders_per_country['us'] if item['id']== 'Q8007')
#wiki_req = requests.get(url_president, headers = {'User-Agent': 'wiki_scraper'}).text
                     

Dropdown(description='Leader:', options=('be Verhofstadt Guy Q12978 https://nl.wikipedia.org/wiki/Guy_Verhofst…

Button(description='Confirm:', style=ButtonStyle())

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [25]:
# 3 lines


from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_req, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 skin-theme-clientpref-day vector-sticky-header-enabled wp25eastereggs-enable-clientpref-0 vector-toc-available" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Prosper Poullet - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-conten

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [26]:
# 2 lines
paragraphs = soup.find_all('p')
for i, p in enumerate(paragraphs):
    print(f'{i}:\n{p.get_text()}')

0:
Prosper Antoine Marie Joseph burggraaf Poullet (Leuven, 5 maart 1868 - aldaar, 3 december 1937) was een Belgisch politicus.
1:
Prosper Poullet was een telg uit de aristocratische familie Poullet, die officieren en magistraten telde. Hij was de zoon van Edmond Poullet (1839-1882) en Pauline Ernst (1840-1903). Zijn vader was advocaat, rechtshistoricus, hoogleraar aan de Katholieke Universiteit Leuven en provincieraadslid van Brabant. Zijn beide grootvaders, Prosper Poullet, voorzitter van de rechtbank van eerste aanleg in Leuven, en Antoine Ernst, rector van de Universiteit van Luik, volksvertegenwoordiger en minister van Justitie, waren hoogleraars aan de Leuvense universiteit.
2:
Zelf huwde hij in 1894 in Leuven met burggravin Marie-Louise de Monge de Franeau (1871-1953), de dochter van een mededocent aan de universiteit, en kleindochter van Constantin Moxhon. Het koppel kreeg zes kinderen.
3:
Hij liep van 1879 tot 1884 middelbare school bij de jozefieten in Leuven en promoveerde in

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [27]:
# <10 lines
first_paragraph = ""
for p in paragraphs:
    text = p.get_text().strip()
    
    if len(text) < 50:
        continue
    
    if '.' not in text:
        continue
    
    first_paragraph = text
    break
print(first_paragraph)

Prosper Antoine Marie Joseph burggraaf Poullet (Leuven, 5 maart 1868 - aldaar, 3 december 1937) was een Belgisch politicus.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [29]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
    response = requests.get(wikipedia_url, headers = {'User-Agent': 'wiki_scraper'})

    soup = BeautifulSoup(response.text, 'html.parser')
    first_paragraph = ""
    for p in soup.find_all('p'):
        text = p.get_text().strip()
    
        if len(text) < 50:
            continue
    
        if '.' not in text:
            continue
    
        first_paragraph = text
        break
    
    return first_paragraph

In [30]:
# Test: 3 lines
print(get_first_paragraph('https://en.wikipedia.org/wiki/Franklin_D._Roosevelt'))
print(get_first_paragraph('https://en.wikipedia.org/wiki/Harry_S._Truman'))
print(get_first_paragraph('https://ru.wikipedia.org/wiki/Путин,_Владимир_Владимирович'))

https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
Franklin Delano Roosevelt[a] (January 30, 1882 – April 12, 1945), also known as FDR, was the 32nd president of the United States, serving from 1933 until his death in 1945. He is the longest-serving U.S. president and the only one to have served more than two terms. His first two terms were centered on combating the Great Depression, while his third and fourth focused on U.S. involvement in World War II. A member of the Democratic Party, Roosevelt served in the New York State Senate from 1911 to 1913 and as the 44th governor of New York from 1929 to 1932.
https://en.wikipedia.org/wiki/Harry_S._Truman
Harry S. Truman[b]  (May 8, 1884 – December 26, 1972) was the 33rd president of the United States, serving from 1945 to 1953. As the 34th vice president in 1945, he assumed the presidency upon the death of Franklin D. Roosevelt that year. Subsequently, Truman implemented the Marshall Plan in the aftermath of World War II to rebuild the ec

### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [31]:
# 3 lines
import re
new_p = re.sub(r"\[.*?\]", "", first_paragraph)
print(new_p)



Prosper Antoine Marie Joseph burggraaf Poullet (Leuven, 5 maart 1868 - aldaar, 3 december 1937) was een Belgisch politicus.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [32]:
# 10 lines
def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
    response = requests.get(wikipedia_url, headers = {'User-Agent': 'wiki_scraper'})

    soup = BeautifulSoup(response.text, 'html.parser')
    first_paragraph = ""
    for p in soup.find_all('p'):
        text = p.get_text().strip()
    
        if len(text) < 50:
            continue
    
        if '.' not in text:
            continue
    
        first_paragraph = re.sub(r"\[.*?\]", "", text)
        break
    
    return first_paragraph

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines



## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [33]:
# < 20 lines
def get_leaders() -> dict:
    # assign root_url and endpoints
    root_url = "https://country-leaders.onrender.com"
    status_url = root_url + "/status"
    countries_url = root_url + "/countries"
    leaders_url = root_url + '/leaders'
    cookie_url = root_url + '/cookie'

    # query the /status endpoint using the get() method and store it in the req variable (1 line)
    req = requests.get(f"{status_url}")
    
    # Query the cookies enpoint, set the cookies variable and display it (2 lines)
    cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
    print('User cookie value:', cookies_val)

    # query the /countries endpoint, assign the output to the countries variable (1 line)
    countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()

    # display the request's status code and the countries variable (1 line)
    print(f"The request for 'countries' status code reply is: {req.status_code}, variable 'countries' return: {countries} ")

    leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}
    
    for val in leaders_per_country.values():
        for d in val:
            d.update({'first_paragraph': get_first_paragraph(d['wikipedia_url'])})
    
    
    return leaders_per_country

get_leaders()

User cookie value: b12eb157-ec67-4736-895e-9ed8ad422e4f
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20',
   'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate':

In [ ]:
# Check the output of your function (2 lines)
output = get_leaders()
print(output)

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [34]:
def get_leaders() -> dict:
    # assign root_url and endpoints
    root_url = "https://country-leaders.onrender.com"
    status_url = root_url + "/status"
    countries_url = root_url + "/countries"
    leaders_url = root_url + '/leaders'
    cookie_url = root_url + '/cookie'

    # query the /status endpoint using the get() method and store it in the req variable (1 line)
    req = requests.get(f"{status_url}")
    
    # Query the cookies enpoint, set the cookies variable and display it (2 lines)
    cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
    print('User cookie value:', cookies_val)

    # query the /countries endpoint, assign the output to the countries variable (1 line)
    countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()

    # display the request's status code and the countries variable (1 line)
    print(f"The request for 'countries' status code reply is: {req.status_code}, variable 'countries' return: {countries} ")

    try:
        leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}
    
        for val in leaders_per_country.values():
            for d in val:
                d.update({'first_paragraph': get_first_paragraph(d['wikipedia_url'])})
    except:
        print(f"Status code for cookies: {requests.get(f"{cookie_url}").status_code}")
        if requests.get(f"{status_url}").status_code != 200:
            print(f"Updating cookies...")
            cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
            print(f'and running the app again...')
            countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()

            leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}

            for val in leaders_per_country.values():
                for d in val:
                    d.update({'first_paragraph': get_first_paragraph(d['wikipedia_url'])})
        
    return leaders_per_country

get_leaders()

User cookie value: 90dedad1-133a-40dd-99e5-1ac97b50b487
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20',
   'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate':

Check the output of your function again.

In [35]:
# Check the output of your function (1 line)
print(get_leaders())

User cookie value: 1c6051e8-14e9-45ab-b00d-2fc3ea7c48ff
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 
{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'deat

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [36]:
# < 20 lines
from requests import Session

def get_first_paragraph(wikipedia_url, session: Session):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
    response = session.get(wikipedia_url, headers = {'User-Agent': 'wiki_scraper'})

    soup = BeautifulSoup(response.text, 'html.parser')
    first_paragraph = ""
    for p in soup.find_all('p'):
        text = p.get_text().strip()
    
        if len(text) < 50:
            continue
    
        if '.' not in text:
            continue
    
        first_paragraph = re.sub(r"\[.*?\]", "", text)
        break
    
    return first_paragraph


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [37]:
# <25 lines
def get_leaders() -> dict:
    # assign root_url and endpoints
    root_url = "https://country-leaders.onrender.com"
    status_url = root_url + "/status"
    countries_url = root_url + "/countries"
    leaders_url = root_url + '/leaders'
    cookie_url = root_url + '/cookie'

    # query the /status endpoint using the get() method and store it in the req variable (1 line)
    req = requests.get(f"{status_url}")
    
    # Query the cookies enpoint, set the cookies variable and display it (2 lines)
    cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
    print('User cookie value:', cookies_val)

    # query the /countries endpoint, assign the output to the countries variable (1 line)
    countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()

    wiki_session = requests.Session()

    # display the request's status code and the countries variable (1 line)
    print(f"The request for 'countries' status code reply is: {req.status_code}, variable 'countries' return: {countries} ")

    try:
        leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}
    
        for val in leaders_per_country.values():
            for d in val:
                d.update({'first_paragraph': get_first_paragraph(d['wikipedia_url'], wiki_session)})
    except:
        print(f"Status code for cookies: {requests.get(f"{cookie_url}").status_code}")
        if requests.get(f"{status_url}").status_code != 200:
            print(f"Updating cookies...")
            cookies_val = requests.get(f"{cookie_url}").cookies.get('user_cookie')
            print(f'and running the app again...')
            countries = requests.get(countries_url, cookies = {"user_cookie": cookies_val}).json()

            leaders_per_country = {country: requests.get(leaders_url, cookies = {"user_cookie": cookies_val},  params = {'country': country}).json() for country in countries}

            for val in leaders_per_country.values():
                for d in val:
                    d.update({'first_paragraph': get_first_paragraph(d['wikipedia_url'], wiki_session)})
        
    return leaders_per_country


Test your new functions.



In [38]:
output = get_leaders()
print(output)

User cookie value: 8b4af186-c5b6-4d80-8a43-8849255e5a8d
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 
{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'deat

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [39]:
# 3 lines
import json
leaders_per_country_dict = get_leaders()
with open('leaders.json', 'w') as f:
    json.dump(leaders_per_country_dict, f)


User cookie value: ab13dff6-fce7-4b1c-8740-ea86e4ad7e6a
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [40]:
# 3 lines
with open('leaders.json') as f:
    d = json.load(f)
    print(d)


{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06', 'first

Make a function `save(leaders_per_country)` to call this code easily.

In [42]:
# 3 lines
#leaders_per_country_dict = get_leaders()
def save_func():
    leaders_per_country_dict = get_leaders()
    with open('leaders.json', 'w') as f:
        json.dump(leaders_per_country_dict, f)

save_func()

User cookie value: 0af52098-2023-4937-926d-fa9982e03832
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 


In [43]:
# Call the function (1 line)
save_func()

User cookie value: b28df9db-e652-4803-853e-bfec4020b8d8
The request for 'countries' status code reply is: 200, variable 'countries' return: ['be', 'us', 'ma', 'fr', 'ru'] 


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!